In [4]:
pip install textBlob

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
#import libraries
import pandas as pd
import numpy as np
from textblob import TextBlob